In [1]:
import open3d as o3d
import numpy as np
import os
import trimesh
from collections import defaultdict
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
os.getcwd()

'/home/piai/SAMPart3D'

In [3]:
segmented = o3d.io.read_point_cloud("/home/piai/SAMPart3D/assets/20_desk_gaussian_segmented.ply")
points = np.asarray(segmented.points)
print(segmented)
print("Points:", points.shape)

PointCloud with 20000 points.
Points: (20000, 3)


In [4]:
o3d.visualization.draw_geometries([segmented])

In [5]:
pcd = o3d.io.read_point_cloud("/home/piai/SAMPart3D/20_desk_gaussian.ply")
points = np.asarray(pcd.points)
print(pcd)
print("Points:", points.shape)

PointCloud with 276384 points.
Points: (276384, 3)


In [6]:
m = o3d.io.read_triangle_mesh("/home/piai/SAMPart3D/20_desk_gaussian.ply")
print("Vertices:", len(m.vertices))
print("Triangles:", len(m.triangles))

[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).
Vertices: 276384
Triangles: 0


In [7]:
o3d.visualization.draw_geometries([pcd])

In [8]:
pts = np.asarray(pcd.points)
z = pts[:, 2]

print("Z min:", z.min())
print("Z max:", z.max())

Z min: -0.22437435388565063
Z max: 0.22444820404052734


### TOP

In [9]:
plane_model, inliers = pcd.segment_plane(
    distance_threshold=0.01,
    ransac_n=3,
    num_iterations=2000
)

[a, b, c, d] = plane_model
print("Plane normal:", a, b, c, d)

tabletop = pcd.select_by_index(inliers)
# rest = pcd.select_by_index(inliers, invert=True)

Plane normal: 0.0003470187265676859 0.0005734955597589277 0.999999775340398 -0.21674714347261123


In [10]:
o3d.io.write_point_cloud("manual_output/part_top.ply", tabletop)

True

In [11]:
top = o3d.io.read_point_cloud("/home/piai/SAMPart3D/manual_output/part_top.ply")

In [12]:
o3d.visualization.draw_geometries([top])

### legs

In [13]:
thickness_leg = 0.0534 #수정 가능 
thickness_frame = 0.03 #수정 가능 
normal_norm = np.sqrt(a*a + b*b + c*c)
distances = np.abs(points @ np.array([a, b, c]) + d) / normal_norm

tabletop_mask_leg = distances < thickness_leg
tabletop_leg = pcd.select_by_index(np.where(tabletop_mask_leg)[0])
legs = pcd.select_by_index(np.where(~tabletop_mask_leg)[0])
tabletop_mask_frame = distances < thickness_frame
tabletop_frame = pcd.select_by_index(np.where(tabletop_mask_frame)[0])
frame = pcd.select_by_index(np.where(~tabletop_mask_frame)[0])

In [14]:
legs_clean, ind = legs.remove_radius_outlier(
    nb_points=30,  
    radius=0.02)

In [15]:
o3d.visualization.draw_geometries([legs_clean])

In [16]:
pts = np.asarray(legs_clean.points)

X = pts[:, :2]   # cluster in XY

db = DBSCAN(
    eps=0.05,      
    min_samples=50
)

labels = db.fit_predict(X)

unique_labels = [l for l in np.unique(labels) if l != -1]
print("Clusters found:", unique_labels)


for i in unique_labels:
    idx = np.where(labels == i)[0]
    part = legs_clean.select_by_index(idx)
    o3d.io.write_point_cloud(f"manual_output/leg_{i+1}.ply", part)

Clusters found: [np.int64(0), np.int64(1), np.int64(2), np.int64(3)]


In [17]:
leg = o3d.io.read_point_cloud("/home/piai/SAMPart3D/manual_output/leg_4.ply")
o3d.visualization.draw_geometries([leg])

### frame

In [18]:
o3d.visualization.draw_geometries([frame])

In [19]:
frame_pts = np.asarray(frame.points)
leg_files = [
    "leg_1.ply",
    "leg_2.ply",
    "leg_3.ply",
    "leg_4.ply"
]

legs = []
for f in leg_files:
    leg = o3d.io.read_point_cloud(f)
    legs.append(leg)

legs_pc = legs[0]
for l in legs[1:]:
    legs_pc += l

leg_pts = np.asarray(legs_pc.points)
nbrs = NearestNeighbors(n_neighbors=1).fit(leg_pts)

leg_dist, _ = nbrs.kneighbors(frame_pts)
leg_clearance = 0.02   # adjust if needed (0.01–0.03 typical)

keep_mask = leg_dist[:, 0] > leg_clearance
frame_no_legs_pts = frame_pts[keep_mask]

frame_no_legs = o3d.geometry.PointCloud()
frame_no_legs.points = o3d.utility.Vector3dVector(frame_no_legs_pts)

o3d.io.write_point_cloud("manual_output/frame_without_legs.ply", frame_no_legs)

True

In [20]:
frame = o3d.io.read_point_cloud("/home/piai/SAMPart3D/manual_output/frame_without_legs.ply")
o3d.visualization.draw_geometries([frame])

In [21]:
frame, _ = frame.remove_radius_outlier(
    nb_points=60,
    radius=0.01)

In [22]:
o3d.visualization.draw_geometries([frame])

In [23]:
points = np.asarray(frame.points)
xy = points[:, :2] 

db = DBSCAN(
    eps=0.025,        
    min_samples=40   
).fit(xy)

labels = db.labels_

unique_labels = set(labels)
print("Clusters:", unique_labels)

Clusters: {np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(-1)}


In [24]:
frame_id = 1 


for lbl in sorted(unique_labels):
    if lbl == -1:
        continue

    idx = np.where(labels == lbl)[0]
    cluster = frame.select_by_index(idx)

    print(f"Saving frame_{frame_id}.ply  (DBSCAN label={lbl}, points={len(idx)})")

    o3d.io.write_point_cloud(
        f"manual_output/frame_{frame_id}.ply",
        cluster)

    frame_id += 1

Saving frame_1.ply  (DBSCAN label=0, points=1977)
Saving frame_2.ply  (DBSCAN label=1, points=5306)
Saving frame_3.ply  (DBSCAN label=2, points=5285)
Saving frame_4.ply  (DBSCAN label=3, points=2163)


In [25]:
frame = o3d.io.read_point_cloud("/home/piai/SAMPart3D/manual_output/frame_4.ply")
o3d.visualization.draw_geometries([frame])